Ejemplo 8.2 del libro
----------

Extraemos una muestra de una distribucion normal y visualizamos de manera interactiva como cambian los valores

In [68]:
#Importamos las librerias necesarias para trabajar
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
from scipy.stats import truncnorm, norm

#Inicializamos la aplicación Dash, para generar los gráficos interactivos
app = dash.Dash(__name__)

# Detallamos los parámetros poblacionales, para construir la distribución
media = 6.5
varianza = 9
desviacion_estandar = np.sqrt(varianza)

# Un día tiene 24 hs. Debemos ajustar los límites para truncar la distribución (0 a 24hs), evitando valores negativos o superiores a 24, en el eje de las abcisas
a, b = 0, 24

#Generamos la distribución truncada.
# 'truncnorm'-- Es una clase de la biblioteca scipy.stats que representa una distribución normal truncada. En este caso, los limites son a y b
# '(a - media) / desviacion_estandar'-- Convierte el límite inferior 'a' en un valor representado en términos de la desviación estándar de la distribución normal estándar
# 'scale=desviacion_estandar'-- Establece la desviación estándar de la distribución truncada. 
trunc_normal = truncnorm(a=(a - media) / desviacion_estandar, 
                         b=(b - media) / desviacion_estandar, 
                         loc=media, 
                         scale=desviacion_estandar)

#Generamos 1000 valores, igualmente espaciados, para calcular y graficar la función de densidad de probabilidad (PDF) de la distribución truncada.
# El limite inferior de esos valores es 0. El limite superior es 19. Se puede ajustar los limites para hacer mas o menos realista la situacion
x = np.linspace(0, 19, 1000)

#Calculamos la función de densidad de probabilidad (PDF)
# PDF es una función que describe la probabilidad de que una variable aleatoria continua tome un valor específico o un rango de valores
pdf = trunc_normal.pdf(x)

#Retomamos el objeto app que instanciamos con dash, para generar los componentes html del grafico interactivo.
app.layout = html.Div([
    html.Div([
        dcc.Graph(id='grafico-distribucion'),
        dcc.Graph(id='grafico-muestra'),
    ], style={'display': 'flex', 'justify-content': 'space-between'}),
    html.Div([
        html.Div("Tamaño de Muestra", style={'margin-top': 20}),
        dcc.Slider(
            id='slider-tamano-muestra',
            min=10,
            max=1000,
            step=10,
            value=100,
            marks={i: str(i) for i in range(10, 1001, 100)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'border': '1px solid #ddd', 'padding': 20, 'margin-bottom': 20}),
    html.Div([
        html.Div("Media Muestral", style={'margin-top': 20}),
        dcc.Slider(
            id='slider-media-muestra',
            min=0,
            max=24,
            step=0.5,
            value=6,
            marks={i: str(i) for i in range(0, 25, 2)},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={'border': '1px solid #ddd', 'padding': 20, 'margin-bottom': 20}),
    html.Div(id='probabilidad-texto', style={'margin-top': 20, 'font-weight': 'bold'}),
    html.Div(id='z-score-texto', style={'margin-top': 20, 'font-weight': 'bold'})
])

#Creamos un callback para escuchar los valores de los sliders y actualizar los graficos
# Los sliders son inputs del sistema y los graficos son los outputs.
@app.callback(
    Output('grafico-distribucion', 'figure'),
    Output('grafico-muestra', 'figure'),
    Output('probabilidad-texto', 'children'),
    Output('z-score-texto', 'children'),
    Input('slider-tamano-muestra', 'value'),
    Input('slider-media-muestra', 'value')
)

#Funcion para actualizar los graficos de la app, con dos parametros.
# Se reciben los valoress de los inputs del callback, se los procesa y se devuelve nuevos valores para los outputs
# El parametro 'tamano_muestra' recibe el valor del slider con el id 'slider-tamano-muestra'. Lo mismo para 'media_muestral'
def actualizar_graficos(tamano_muestra, media_muestral):
    
    # El método rvs del objeto truncnorm de scipy se usa para generar valores aleatorios (simulados) de la distribucion truncada
    muestra = trunc_normal.rvs(size=tamano_muestra)
    
    #Creamos la linea azul parar representar la distribucion normal truncada de la variable en la poblacion
    line_data = go.Scatter(
        x=x,
        y=pdf,
        mode='lines',
        line=dict(color='blue'),
        name=f'Dist. Truncada({media}, {varianza})'
    )
    
    #Creamos una linea punteada verde para la media poblacional
    media_poblacional_line = go.Scatter(
        x=[media, media],
        y=[0, max(pdf)],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name=f'Media Poblacional: {media}'
    )

    #Creamos una linea punteada roja para el slide de la media muestral
    mean_line = go.Scatter(
        x=[media_muestral, media_muestral],
        y=[0, max(pdf)],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name=f'Media Muestral: {media_muestral}'
    )

    #Calculamos la probabilidad de obtener una muestra con media >= media_muestral, usando la fomula del puntaje z
    # La función 'norm.cdf(z_score)' calcula la probabilidad acumulada hasta el valor z_score en una distribución normal estándar (media 0 y desviación estándar 1).
    # '1 - norm.cdf(z_score)' nos da el area bajo la curva de la distribución normal a la derecha del valor z_score. Multiplicamos luego por 100 para convertir en porcentaje
    z_score = (media_muestral - media) / (desviacion_estandar / np.sqrt(tamano_muestra))
    probabilidad = (1 - norm.cdf(z_score))*100
    probabilidad_texto = f'Probabilidad de media muestral mayor o igual a {media_muestral}: {probabilidad:.4f}%'
    z_score_texto = f'Fórmula del puntaje z: ({media_muestral} - {media}) / ({desviacion_estandar} / {np.sqrt(tamano_muestra)}) = {z_score:.4f}'

    #Definimos las caracteristicas del grafico poblacional siguiendo las reglas de la biblioteca plotly
    layout_distribucion = go.Layout(
        title='Distribución Normal Truncada',
        xaxis=dict(title='Horas de descanso'),
        yaxis=dict(title='Densidad de probabilidad'),
        showlegend=True
    )

    #Creamos un histograma para la muestra, segun el 'n' definido por el slide
    #Seteamos el numero de bins en 40 para agrupar en intervalos de 0,5
    #El eje de las ordenadas mide la frecuencia relativa
    hist_data = go.Histogram(
        x=muestra,
        nbinsx=40,
        name='Muestra',
        marker_color='green',
        opacity=0.7,
        histnorm='percent'
    )

    #Definimos las caracteristicas del grafico muestral, pasando dinamicamente el tamano de la muestra y el valor de la media establecido en el slide
    layout_muestra = go.Layout(
        title=f'Distribución de la Muestra con Tamaño {tamano_muestra} y Media {media_muestral}',
        xaxis=dict(title='Horas de descanso'),
        yaxis=dict(title='Frecuencia relativa'),
        showlegend=False
    )

    #Funcion return para devolver los graficos y las oraciones que explican el calculo
    return (
        go.Figure(data=[line_data, media_poblacional_line, mean_line], layout=layout_distribucion),
        go.Figure(data=[hist_data], layout=layout_muestra),
        probabilidad_texto,z_score_texto
    )

#Ejecutamos la aplicacion
if __name__ == '__main__':
    app.run_server(debug=True)
